In [1]:
import pickle
from encode_utils.rerank_data import rerank_dist, rerank_single
from encode_utils.efficient_rerank import get_effrerank_model, run_comstyle
from encode_utils.sco_funct import weightaddprob, default_scofunct
from encode_utils.mt_scores import get_scores_auto
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import torch
import random
import os
import re
from encode_utils.new_mask_utils import randomsingle, useall
from encode_utils.eval_utils import all_lattice_multi, mean, all_unnoun_multi
from generate_tables import metrics_mapping
from encode_utils.rerank_data import rerank_df, rerank_single, rerank_rand, rerank_weighted
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

2022-12-29 08:05:56.892611: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-29 08:05:56.892637: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# run pipeline for generating lattice preds, using different parameters

In [3]:
# set up information for set
col = {
    "noun_xsum": ["nounsum_reversed/", "nounxsumlargeexplodev2.csv"],
    "noun_fren": ["frtest_reversed/", "nounlargeexplodev1.csv"],
    "mt_fren": ["frtest_reversed/", "frenchlargeexplodev1.csv"],
    "mt_ende": ["detest_reversed/", "germanlargeexplodev1.csv"],
    "mt_enru": ["rutest_reversed/", "russianlargeexplodev1.csv"],
    "mt_fren_b12": ['reversed_mtfren_beam12/', 'mtfrenbeam12v2.csv'],
    "mt_fren_b50": ['reversed_mtfren_beam50/', 'mtfrenbeam50v2.csv'],
    "noun_xsum_b12": ['reversed_xsum_beam12/', 'nounxsumbeam12v2.csv'],
    "noun_xsum_b50": ['reversed_xsum_beam50/', 'nounxsumbeam50v2.csv'],
}
curcol = "noun_xsum_b12"
gsuffix = col[curcol][0]
expl_fname = col[curcol][1]
base = "outputs/graph_pickles/"+gsuffix
goldmetric = "utnoun"
explode_df = pd.read_csv("outputs/score_csvs/"+expl_fname)
#TODO switch back for french-english
if "fren" in curcol:
    SETLEN = len(os.listdir(base))
else:
    SETLEN = int(len(os.listdir(base))/2)

In [4]:
# use noun model
if "noun" in expl_fname:
    encodemod = get_effrerank_model("noun")
# use mt model (causal)
else:
    encodemod = get_effrerank_model("comstyle")
xlm_tok = AutoTokenizer.from_pretrained("xlm-roberta-base")

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.
Loading weights from /mnt/data1/prasann/latticegen/lattice-generation/COMET/lightning_logs/version_44/checkpoints/epoch=9-step=40000.ckpt.


freeze embeds


In [5]:
SETLEN = len(os.listdir(base))

In [6]:
args = {
    'setlen':int(SETLEN),
    'tok':xlm_tok, 
    'dev':device,
    'model':encodemod,
    'explode_df':explode_df,
    'base':base,
    'goldmetric':goldmetric,
    'device':device, 
    'efficient':False,
    'noregen':False
}

In [7]:
latpreds = all_lattice_multi(1, default_scofunct, randomsingle, args)

584   584   tensor(0.8009, device='cuda:2', grad_fn=<SumBackward0>)   tensor(0.8009, device='cuda:2', grad_fn=<SumBackward0>)  
585   585   tensor(0.5751, device='cuda:2', grad_fn=<SumBackward0>)   tensor(0.5751, device='cuda:2', grad_fn=<SumBackward0>)  
586   586   tensor(0.9163, device='cuda:2', grad_fn=<SumBackward0>)   tensor(0.9163, device='cuda:2', grad_fn=<SumBackward0>)  
587   587   tensor(0.7515, device='cuda:2', grad_fn=<SumBackward0>)   tensor(0.7515, device='cuda:2', grad_fn=<SumBackward0>)  
588   588   tensor(0.7310, device='cuda:2', grad_fn=<SumBackward0>)   tensor(0.7310, device='cuda:2', grad_fn=<SumBackward0>)  
589   589   tensor(0.8298, device='cuda:2', grad_fn=<SumBackward0>)   tensor(0.8298, device='cuda:2', grad_fn=<SumBackward0>)  
590   590   tensor(0.8404, device='cuda:2', grad_fn=<SumBackward0>)   tensor(0.8404, device='cuda:2', grad_fn=<SumBackward0>)  
591   591   tensor(1.2070, device='cuda:2', grad_fn=<SumBackward0>)   tensor(1.2070, device='cuda:2', gr

In [33]:
len(explode_df[explode_df['ref']==worseuns[7]])

12

In [42]:
latpreds['gold'][0] # maybe there's a tokenization mismatch? 

'Portsmouth moved into the League Two play-off places with victory at Notts County thanks to two goals from Luke Bennett.'

In [57]:
ind = 6
print(latpreds['gold'][ind], " ", latpreds['goldsco'][ind])
print(latpreds['hyp'][ind], " ", latpreds['hypsco'][ind])

Plans for two new "metro mayors" in Yorkshire are set to be unveiled by the chancellor.   tensor(0.5141, device='cuda:2', grad_fn=<SumBackward0>)
Plans for two new " metro mayors " in Yorkshire are set to be unveiled by the chancellor.   tensor(0.5366, device='cuda:2', grad_fn=<SumBackward0>)


In [ ]:
for l in list(latpreds['hyp']):
    if l not in explode_df['hyp']:
        print("o")
    else:
        printn("i")

In [30]:
worseuns = latpreds[latpreds['hypsco']<latpreds['goldsco']]['ref'].unique()

In [ ]:
graph = pickle.load(open(base+str(0), 'rb'))

In [27]:
graph['root'].nextlist[1].token_str

'Two'

In [ ]:
graph[3]

In [16]:
(len(explode_df)/1000)/(5*32)

4.2108125

In [18]:
tind = 1
print(latpreds.loc[tind])
explode_df[explode_df['hyp']==latpreds['gold'][tind]]

hyp         Negotiations with Iran on its nuclear programm...
hypsco      tensor(0.7139, device='cuda:2', grad_fn=<SumBa...
gold        Negotiations with Iran on its nuclear programm...
goldsco     tensor(0.7139, device='cuda:2', grad_fn=<SumBa...
ascos                                    [0.7138539552688599]
ahyps       [Negotiations with Iran on its nuclear program...
numnodes                                                  174
src         Les négociations avec l’Iran sur son programme...
ref         As a result, the negotiations with Iran over i...
Name: 1, dtype: object


,Unnamed: 0,src,ref,hyp,dcqeold,comet,cqe,posthoc,dupcqe
4946,4946,Les négociations avec l’Iran sur son programme...,"As a result, the negotiations with Iran over i...",Negotiations with Iran on its nuclear programm...,0.787222,0.807614,0.641687,0.347093,0.764838


In [8]:
print(mean(latpreds['goldsco']))
print(mean(latpreds['hypsco']))
print(mean(latpreds['goldsco'])-mean(latpreds['hypsco']))

tensor(0.8481, device='cuda:2', grad_fn=<DivBackward0>)
tensor(0.8480, device='cuda:2', grad_fn=<DivBackward0>)
tensor(6.0797e-05, device='cuda:2', grad_fn=<SubBackward0>)


In [ ]:
goldsco

In [6]:
metrics_mapping("comet", lpredstmp)

wmt20-comet-da is already in cache.
Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.


freeze embeds


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
Predicting DataLoader 0: 100%|████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:04<00:00,  3.28it/s]


TOOK TIME  8.24


In [7]:
mean(lpredstmp['comet'])

0.6508571936298679

In [49]:
lpredstmp = pd.read_csv("outputs/predcsvs/noun_comstyle_xsum_randsing_8shot.csv")

In [41]:
luns = latpreds['ref'].unique()

In [42]:
edf = explode_df[explode_df['ref'].isin(luns)]

In [ ]:
olatpreds = pd.read_csv()

In [12]:
rerank_df(explode_df, rerank_single, ['comet', 'comet'])

0.7609684539601108

In [9]:
gs = list(latpreds['hypsco'])
hs = list(latpreds['goldsco'])

In [10]:
mean([max(gs[i], hs[i]) for i in range(len(latpreds))])

tensor(0.7450, device='cuda:2', grad_fn=<DivBackward0>)

In [ ]:
metrics_mapping

In [9]:
outfile = "mt_fren_eelb501shotweighted.csv"
latpreds.to_csv("outputs/predcsvs/"+outfile)

In [5]:
lpredstmp = pd.read_csv("outputs/predcsvs/"+"mt_fren_eelb501shotweighted.csv")

In [7]:
import time

In [11]:
phoc = all_unnoun_multi(1, "dupcqe", args)
print(mean(phoc))


426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
tensor(0.6634, device='cuda:2', grad_fn=<DivBackward0>)


In [ ]:
s = time.time()
args['noregen'] = True
args['setlen'] = 100
phoc = all_unnoun_multi(1, "utnoun", args)
print(mean(phoc))
e = time.time()

In [11]:
(e-s)/len(phoc)

0.9002537321537099

In [ ]:
args['explode_df']

In [37]:
explode_df['phocrank'] = explode_df['posthoc']*-1
#rand = all_unnoun_multi(32)

In [32]:
mean(rand)

tensor(0.8749, device='cuda:0', grad_fn=<DivBackward0>)

In [49]:
latpreds[latpreds['goldsco']-latpreds['hypsco']>.2]

,hyp,hypsco,gold,goldsco,ascos,ahyps,numnodes
7,Немецкие и французские стороны явно нуждались ...,"tensor(0.3987, device='cuda:0', grad_fn=<SumBa...",Германия и Франция явно нуждались во взаимном ...,"tensor(0.9256, device='cuda:0', grad_fn=<SumBa...",[0.3987096846103668],[Немецкие и французские стороны явно нуждались...,385
13,Небольшая часть населения Индии не имеет ни од...,"tensor(0.6347, device='cuda:0', grad_fn=<SumBa...",Сложно понять низкую позицию Индии.,"tensor(0.9250, device='cuda:0', grad_fn=<SumBa...",[0.6347033977508545],[Небольшая часть населения Индии не имеет ни о...,501
19,Так много о предсказках Кругмана.,"tensor(0.2668, device='cuda:0', grad_fn=<SumBa...","Это все, что предсказывал Кругман.","tensor(0.7456, device='cuda:0', grad_fn=<SumBa...",[0.2667738199234009],[Так много о предсказках Кругмана.],554
31,"Мы узнали, что это была просто воздушная пузыр...","tensor(0.8288, device='cuda:0', grad_fn=<SumBa...","Мы поняли, что это было просто кружево.","tensor(1.0661, device='cuda:0', grad_fn=<SumBa...",[0.8288499116897583],"[Мы узнали, что это была просто воздушная пузы...",558
71,В некоторых странах до 30% более оптимистичных...,"tensor(0.7622, device='cuda:0', grad_fn=<SumBa...",До 30% наиболее оптимистичных потребителей в н...,"tensor(1.0325, device='cuda:0', grad_fn=<SumBa...",[0.7622228264808655],[В некоторых странах до 30% более оптимистичны...,381


In [ ]:
# code for getting comparable sets for lattice generation
#frset = pd.read_csv("outputs/score_csvs/frenchlargeexplodev1.csv")
#xsumset = pd.read_csv("outputs/score_csvs/nounxsumlargeexplodev2.csv")
#frset = frset.drop_duplicates(subset='ref')
#xsumset = xsumset.drop_duplicates(subset='ref')
#xsumset.to_csv("sumexamps.csv")
#frset.to_csv("frset.csv")